In [ ]:
from breeze_connect import BreezeConnect
import json
# Initialize SDK
breeze = BreezeConnect(api_key="05l3(7895X4136976h@Za90m09681999")

# Generate Session
breeze.generate_session(api_secret="516o05v3500859%I515F02eo#3l27876",
                        session_token="4757966225")


In [ ]:
import json
import pyspark
import pandas as pd 
from pyspark.sql import SparkSession
import functools 
  
# explicit function 
def unionAll(dfs): 
    return functools.reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), dfs) 

sc = pyspark.SparkContext('local')
option = breeze.get_trade_list(from_date="2024-04-01T06:00:00.000Z",
                        to_date="2024-09-23T06:00:00.000Z",
                        exchange_code="NSE",
                        product_type="options",
                        action="",
                        stock_code="")

ostring = json.dumps(option, indent=4)  
dfo = spark.read.json(sc.parallelize([ostring]))
#dfo.show(truncate=False)
flat_dfo = dfo.select(explode("Success").alias("Success"))
#flat_dfo.show(truncate=False)
#flat_dfo.printSchema()
#json_ostring = json.loads(ostring)
#print(type(flat_dfo))

optionplus= breeze.get_trade_list(from_date="2024-04-01T06:00:00.000Z",
                        to_date="2024-09-23T06:00:00.000Z",
                        exchange_code="NSE",
                        product_type="optionplus",
                        action="",
                        stock_code="")

opstring = json.dumps(optionplus, indent=4)  
dfop = spark.read.json(sc.parallelize([opstring]))
flat_dfop = dfop.select(explode("Success").alias("Success"))
#flat_dfop.printSchema()

unioned_df = unionAll([flat_dfop, flat_dfo]) 
unioned_df.printSchema()
#flat_dfop.show(truncate=False)


In [ ]:
import json
import pandas as pd 
import pyspark 
from pyspark.sql.functions import explode
from pyspark.sql import SparkSession 


BankNifty =json.dumps (breeze.get_quotes(stock_code="CNXBAN",
                                          exchange_code="NSE",
                                          expiry_date="",
                                          product_type="cash",
                                          right="others",
                                          strike_price="0"
                                        )
                       )

spark = SparkSession.builder.appName('sparkdf').getOrCreate()                        
sc = spark.sparkContext                       
bndf = spark.read.json(sc.parallelize([BankNifty])) 

flat_bndf = bndf.select("Error","Status",explode("Success").alias("Success"))
flat_bndf =  flat_bndf.select (flat_bndf.Success.ltp.alias("now"),"Success.open","Success.low","Success.high",flat_bndf.Success.previous_close.alias("lastdayclose")).where(flat_bndf.Success.ltp > 0)  
flat_bndf.show()
#flat_bndf.show()